In [67]:
import pandas as pd
import os
import glob

In [68]:
results_path = "/home/schmied.christopher/FMP_Docs/Projects/eu_os_ecbl_qc/results/QC_Results/"

# USC

In [69]:
usc_qc = pd.read_csv(os.path.join(results_path, "USC_QC_Summary.csv"))

In [70]:
usc_first_run = usc_qc[usc_qc["run"] == "First Run"]
usc_repeat = usc_qc[usc_qc["run"] == "Repeat"]
usc_repeat2 = usc_qc[usc_qc["run"] == "Repeat2"]
usc_first_run_reject = usc_first_run[usc_first_run["qc_summary"] == "Repeat"]

In [71]:
# Check first run for duplicate plate IDs
duplicate_plate_ids = usc_first_run[usc_first_run.duplicated(subset="plate_map_name", keep=False)]
len(duplicate_plate_ids)

if len(duplicate_plate_ids) > 0:
    print("Duplicate plate IDs found in first run:")
    print(duplicate_plate_ids[["plate_map_name", "qc_summary", "run"]])
else:
    print("No duplicate plate IDs found in first run.")    

No duplicate plate IDs found in first run.


In [72]:
duplicate_plate_ids_repeat = usc_repeat[usc_repeat.duplicated(subset="plate_map_name", keep=False)]
len(duplicate_plate_ids_repeat)

if len(duplicate_plate_ids_repeat) > 0:
    print("Duplicate plate IDs found in repeat run:")
    print(duplicate_plate_ids_repeat[["plate_map_name", "qc_summary", "run"]])
else:
    print("No duplicate plate IDs found in repeat run.")  

No duplicate plate IDs found in repeat run.


In [73]:
# After analysis we agreed on a set of repeats (labeled as repeats in run)
usc_first_run["qc_summary"] = usc_first_run["plate_map_name"].isin(usc_repeat["plate_map_name"]).map({True: "repeat", False: "pass"})

# After repeate run there was a second repeat run
usc_repeat["qc_summary"] = usc_repeat["plate_map_name"].isin(usc_repeat2["plate_map_name"]).map({True: "repeat", False: "pass"})

# I know that all plates passed in the second repeat run, so I can directly set them to pass
usc_repeat2["qc_summary"] = "pass"

usc_qc_updated = pd.concat([usc_first_run, usc_repeat, usc_repeat2], ignore_index=True)

In [74]:
cols = ["heatmap", "cell_number", "comment", "intra_plate", "intra_batch", "inter_batch", "qc_summary"]
usc_qc_updated[cols] = usc_qc_updated[cols].apply(lambda x: x.str.lower())
usc_qc_updated.to_excel(os.path.join(results_path, "USC_QC_Summary_Updated.xlsx"), index=False)

In [ ]:
# Create table relevant for martin
usc_qc_updated_martin = usc_qc_updated.copy()
usc_qc_updated_martin["batch_date"] = pd.to_datetime(usc_qc_updated_martin["batch_date"], errors="coerce")
usc_qc_updated_martin = usc_qc_updated_martin[usc_qc_updated_martin["batch_date"].notna()]
usc_qc_updated_martin["batch_date_YYMMDD"] = usc_qc_updated_martin["batch_date"].dt.strftime("%y%m%d")
usc_qc_updated_martin["plate_ID_martin"] = usc_qc_updated_martin["plate"] + usc_qc_updated_martin["replicate"] + usc_qc_updated_martin["batch_date_YYMMDD"]

usc_qc_updated_martin_pass = usc_qc_updated_martin[usc_qc_updated_martin["qc_summary"] == "pass"]
usc_qc_updated_martin_repeat = usc_qc_updated_martin[usc_qc_updated_martin["qc_summary"] == "repeat"]

usc_qc_updated_martin_pass_reduced = usc_qc_updated_martin_pass[["plate_ID_martin", "plate_map_name", "replicate", "plate", "batch_date", "qc_summary"]]
usc_qc_updated_martin_pass_reduced["batch_date"] = usc_qc_updated_martin_pass_reduced["batch_date"].dt.strftime("%Y-%m-%d")
usc_qc_updated_martin_repeat_reduced = usc_qc_updated_martin_repeat[["plate_ID_martin", "plate_map_name", "replicate", "plate", "batch_date", "qc_summary"]]
usc_qc_updated_martin_repeat_reduced["batch_date"] = usc_qc_updated_martin_repeat_reduced["batch_date"].dt.strftime("%Y-%m-%d")

# get only the plate IDs for martin
usc_qc_updated_martin_pass_reduced.to_excel(os.path.join(results_path, "USC_QC_Pass_Plate_IDs_Martin.xlsx"), index=False)
usc_qc_updated_martin_repeat_reduced.to_excel(os.path.join(results_path, "USC_QC_Rejects_Plate_IDs_Martin.xlsx"), index=False)

# MEDINA

In [97]:
medina_qc = pd.read_csv(os.path.join(results_path, "MEDINA_QC_Summary.csv"))

In [98]:
medina_first_run = medina_qc[medina_qc["run"] == "First Run"]
medina_second_run = medina_qc[medina_qc["run"] == "Second Run"]

medina_all_runs = medina_qc[medina_qc["run"].isin(["First Run", "Second Run"])]

medina_repeat = medina_qc[medina_qc["run"] == "Repeat"]
medina_unclear = medina_qc[medina_qc["run"] == "Unclear"]

medina_all_runs_rejects = medina_all_runs[medina_all_runs["qc_summary"] == "Repeat"]

In [99]:
# Check first run for duplicate plate IDs
duplicate_plate_ids_first_run = medina_first_run[medina_first_run.duplicated(subset="plate_map_name", keep=False)]
len(duplicate_plate_ids_first_run)

if len(duplicate_plate_ids_first_run) > 0:
    print("Duplicate plate IDs found in first run:")
    print(duplicate_plate_ids[["plate_map_name", "qc_summary", "run"]])
else:
    print("No duplicate plate IDs found in first run.")    

No duplicate plate IDs found in first run.


In [100]:
# Check first run for duplicate plate IDs
duplicate_plate_ids_second_run = medina_second_run[medina_second_run.duplicated(subset="plate_map_name", keep=False)]
len(duplicate_plate_ids_second_run)

if len(duplicate_plate_ids_second_run) > 0:
    print("Duplicate plate IDs found in second run:")
    print(duplicate_plate_ids_second_run[["plate_map_name", "qc_summary", "run"]])
else:
    print("No duplicate plate IDs found in second run.")    

No duplicate plate IDs found in second run.


In [101]:
# Check first run for duplicate plate IDs
duplicate_plate_ids_all_runs = medina_all_runs[medina_all_runs.duplicated(subset="plate_map_name", keep=False)]
len(duplicate_plate_ids_all_runs)

if len(duplicate_plate_ids_all_runs) > 0:
    print("Duplicate plate IDs found in all runs:")
    print(duplicate_plate_ids_all_runs[["plate_map_name", "qc_summary", "run"]])
else:
    print("No duplicate plate IDs found in all runs.") 

No duplicate plate IDs found in all runs.


In [102]:
# Check first run for duplicate plate IDs
duplicate_plate_ids_repeat = medina_repeat[medina_repeat.duplicated(subset="plate_map_name", keep=False)]
len(duplicate_plate_ids_repeat)

if len(duplicate_plate_ids_repeat) > 0:
    print("Duplicate plate IDs found in repeat run:")
    print(duplicate_plate_ids_repeat[["plate_map_name", "qc_summary", "run"]])
else:
    print("No duplicate plate IDs found in repeat run.") 

No duplicate plate IDs found in repeat run.


In [103]:
# After analysis we agreed on a set of repeats (labeled as repeats in run)
medina_all_runs["qc_summary"] = medina_all_runs["plate_map_name"].isin(medina_repeat["plate_map_name"]).map({True: "repeat", False: "pass"})

In [104]:
# I know that all plates passed in the second repeat run, so I can directly set them to pass
medina_repeat["qc_summary"] = "pass"

In [105]:
medina_qc_updated = pd.concat([medina_all_runs, medina_repeat, medina_unclear], ignore_index=True)

In [106]:
cols = ["heatmap", "cell_number", "intra_plate", "intra_batch", "inter_batch", "qc_summary"]
medina_qc_updated[cols] = medina_qc_updated[cols].apply(lambda x: x.str.lower())
medina_qc_updated.to_excel(os.path.join(results_path, "Medina_QC_Summary_Updated.xlsx"), index=False)

In [108]:
# Create table relevant for martin
medina_qc_updated_martin = medina_qc_updated.copy()
medina_qc_updated_martin["batch_date"] = pd.to_datetime(medina_qc_updated_martin["batch_date"], errors="coerce")
medina_qc_updated_martin = medina_qc_updated_martin[medina_qc_updated_martin["batch_date"].notna()]
medina_qc_updated_martin["batch_date_YYMMDD"] = medina_qc_updated_martin["batch_date"].dt.strftime("%y%m%d")
medina_qc_updated_martin["plate_ID_martin"] = medina_qc_updated_martin["plate"] + medina_qc_updated_martin["replicate"] + medina_qc_updated_martin["batch_date_YYMMDD"]

medina_qc_updated_martin_pass = medina_qc_updated_martin[medina_qc_updated_martin["qc_summary"] == "pass"]
medina_qc_updated_martin_repeat = medina_qc_updated_martin[medina_qc_updated_martin["qc_summary"] == "repeat"]

medina_qc_updated_martin_pass_reduced = medina_qc_updated_martin_pass[["plate_ID_martin", "plate_map_name", "replicate", "plate", "batch_date", "qc_summary"]]
medina_qc_updated_martin_pass_reduced["batch_date"] = medina_qc_updated_martin_pass_reduced["batch_date"].dt.strftime("%Y-%m-%d")
medina_qc_updated_martin_repeat_reduced = medina_qc_updated_martin_repeat[["plate_ID_martin", "plate_map_name", "replicate", "plate", "batch_date", "qc_summary"]]
medina_qc_updated_martin_repeat_reduced["batch_date"] = medina_qc_updated_martin_repeat_reduced["batch_date"].dt.strftime("%Y-%m-%d")

# get only the plate IDs for martin
medina_qc_updated_martin_pass_reduced.to_excel(os.path.join(results_path, "Medina_QC_Pass_Plate_IDs_Martin.xlsx"), index=False)
medina_qc_updated_martin_repeat_reduced.to_excel(os.path.join(results_path, "Medina_QC_Rejects_Plate_IDs_Martin.xlsx"), index=False)